In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import joblib

# Load your dataset
df = pd.read_csv('/content/drive/MyDrive/data/user_rating_clean.csv')

# Create a matrix of user-item ratings
user_item_matrix = df.pivot_table(index='User_Id', columns='Place_Id', values='Place_Ratings', fill_value=0)

# Normalize the ratings for collaborative filtering
user_item_matrix_normalized = user_item_matrix.sub(user_item_matrix.mean(axis=1), axis=0)

# Split the dataset into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.2)

# Create user-item matrices for training and testing sets
train_user_item_matrix = train_data.pivot_table(index='User_Id', columns='Place_Id', values='Place_Ratings', fill_value=0)
test_user_item_matrix = test_data.pivot_table(index='User_Id', columns='Place_Id', values='Place_Ratings', fill_value=0)

# Train-test split for neural network
train_nn, test_nn = train_test_split(user_item_matrix_normalized.values, test_size=0.2)

# Function to predict ratings using cosine similarity
def predict_ratings(user_item_matrix, similarity_matrix):
    # Calculate the dot product of the similarity matrix and the user-item matrix
    pred = similarity_matrix.dot(user_item_matrix)
    pred /= np.array([np.abs(similarity_matrix).sum(axis=1)]).T
    return pred

# Function to calculate RMSE
def rmse(prediction, actual):
    prediction = prediction[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, actual))

# Build a more complex Neural Network for model
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(units=128, activation=tf.nn.relu, input_dim=train_nn.shape[1]))
model.add(tf.keras.layers.Dense(units=256, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(units=128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(units=train_nn.shape[1], activation='linear'))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model for 100 epochs
num_epochs = 100
for epoch in range(num_epochs):

    model.fit(train_nn, train_nn, epochs=1, validation_data=(test_nn, test_nn), verbose=0)
    # Calculate user similarity using cosine similarity
    user_similarity = cosine_similarity(train_user_item_matrix)

    # Make rating predictions for travel destinations
    user_item_prediction = predict_ratings(train_user_item_matrix.values, user_similarity)

    # Calculate and print RMSE on the training data
    train_rmse = rmse(user_item_prediction, train_user_item_matrix.values)
    print(f'Epoch {epoch + 1}/{num_epochs} - Train RMSE: {train_rmse}')

# Get the predicted ratings for all users
all_users_ratings = pd.DataFrame(user_item_prediction, index=train_user_item_matrix.index, columns=train_user_item_matrix.columns)


Epoch 1/100 - Train RMSE: 2.8382771444117587
Epoch 2/100 - Train RMSE: 2.8382771444117587
Epoch 3/100 - Train RMSE: 2.8382771444117587
Epoch 4/100 - Train RMSE: 2.8382771444117587
Epoch 5/100 - Train RMSE: 2.8382771444117587
Epoch 6/100 - Train RMSE: 2.8382771444117587
Epoch 7/100 - Train RMSE: 2.8382771444117587
Epoch 8/100 - Train RMSE: 2.8382771444117587
Epoch 9/100 - Train RMSE: 2.8382771444117587
Epoch 10/100 - Train RMSE: 2.8382771444117587
Epoch 11/100 - Train RMSE: 2.8382771444117587
Epoch 12/100 - Train RMSE: 2.8382771444117587
Epoch 13/100 - Train RMSE: 2.8382771444117587
Epoch 14/100 - Train RMSE: 2.8382771444117587
Epoch 15/100 - Train RMSE: 2.8382771444117587
Epoch 16/100 - Train RMSE: 2.8382771444117587
Epoch 17/100 - Train RMSE: 2.8382771444117587
Epoch 18/100 - Train RMSE: 2.8382771444117587
Epoch 19/100 - Train RMSE: 2.8382771444117587
Epoch 20/100 - Train RMSE: 2.8382771444117587
Epoch 21/100 - Train RMSE: 2.8382771444117587
Epoch 22/100 - Train RMSE: 2.83827714441175

In [ ]:
# Hitung rata-rata rating untuk setiap tempat wisata
average_place_ratings = df.groupby('Place_Id')['Place_Ratings'].mean()

# Hitung jumlah rating untuk setiap tempat wisata
count_place_ratings = df['Place_Id'].value_counts()

# Gabungkan informasi rata-rata rating dan jumlah rating ke dalam satu DataFrame
place_info = pd.DataFrame({'Average_Rating': average_place_ratings, 'Rating_Count': count_place_ratings})

# Pilih jumlah rekomendasi yang diinginkan
top_n = 10

# Pilih tempat wisata dengan rating tertinggi dan jumlah rating terbanyak
top_recommendations = place_info.sort_values(by=['Average_Rating', 'Rating_Count'], ascending=[False, False]).head(top_n)

# Tampilkan informasi tempat wisata yang direkomendasikan
recommended_places_info = df[df['Place_Id'].isin(top_recommendations.index)][['Place_Id', 'Place_Name', 'Category', 'City']].drop_duplicates()
recommended_places_info = recommended_places_info.set_index('Place_Id')
recommended_places_info = recommended_places_info.loc[top_recommendations.index]

print(f'\nTop {top_n} Recommendations based on Rating and Rating Count:\n')
print(recommended_places_info)



Top 10 Recommendations based on Rating and Rating Count:

                                  Place_Name       Category              City
477                Curug Ciparay Tasikmalaya     Cagar Alam  Kota Tasikmalaya
518                Taman Rekreasi Wiladatika  Taman Hiburan             Depok
500                   CIMAHI Convention Hall         Budaya            Cimahi
441                       Alun-alun Sumedang  Taman Hiburan          Sumedang
493                 Cagar Budaya Pulo Majeti     Cagar Alam            Banjar
476                   Taman Kota Tasikmalaya  Taman Hiburan  Kota Tasikmalaya
515                Taman Herbal Insani Depok  Taman Hiburan             Depok
449  Kampung Karuhun ECO Green Park Sumedang         Budaya          Sumedang
517                                Godongijo     Cagar Alam             Depok
519                          Setu Pengasinan     Cagar Alam             Depok


In [ ]:
# Save the model to an h5 file
model.save("/content/drive/MyDrive/data/fitur1-2.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
